## 通过深度学习框架简洁地实现线性回归模型

In [90]:
%matplotlib inline
import numpy as np
import torch
from torch.utils import data 

1.先导入linReg_pytorch文件中已实现(也可从d2l库中导入，但我安装不上，放弃了)

In [91]:
def synthetic_data(w, b, num_examples): # 人造数据集
    X = torch.normal(0, 1, (num_examples, len(w))) # 均值为0方差为1的服从正态分布的随机数;
    # 尺寸为 num_examples行 len(w)列; (1000,2)
    y = torch.matmul(X,w) + b # (1000,2) * (2,1) + (1)
    y += torch.normal(0, 0.01, y.shape) # 添加噪声

    return X, y.reshape((-1,1)) # 将y做成一个列向量(1000,1)返回

In [92]:
true_w = torch.tensor([2,-3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000) # 生成数据

2.利用torch自动创建生成小批量数据

In [93]:
def load_array(data_arrays, batch_size, is_train=True):
    # 构造一个PyTorch数据迭代器
    dataset = data.TensorDataset(*data_arrays) # 拿到数据集 *将tuple拆包
    return data.DataLoader(dataset, batch_size, shuffle=is_train) # 每次从中随机挑选batch_size个样本
    # TensorDataset将输入的两类数据一一对应； DataLoader将两类数据重新排序

batch_size = 10
data_iter = load_array((features, labels), batch_size) # 此处直接调用torch.utils中的data库，生成类似迭代器
# type(data_iter) # torch.utils.data.dataloader.DataLoader
# next(iter(data_iter)) # 转换为Python的可迭代对象;并输出一个
for X, y in data_iter:
    print(X,'\n', y)
    break

tensor([[-0.2154, -0.7530],
        [ 0.0838, -1.3488],
        [ 0.9086,  1.0373],
        [ 0.6088,  1.1608],
        [ 0.1283, -0.4211],
        [ 1.0195, -1.3560],
        [-1.7467,  0.6025],
        [ 1.2381, -0.5618],
        [ 0.1635, -0.5379],
        [-0.1785, -0.8679]]) 
 tensor([[ 6.3234],
        [ 8.9543],
        [ 2.4828],
        [ 1.4773],
        [ 5.8938],
        [10.8498],
        [-1.3516],
        [ 8.5833],
        [ 6.3558],
        [ 6.8122]])


3. 定义模型

In [94]:
from torch import nn # neural network 定义了大量的层

net = nn.Sequential(nn.Linear(2,1)) # 用Linear层(线性层/全连接层)；输入维度2 输出维度1
# 然后将Linear层放到Sequential容器中，该容器理解为List of layers，将层按序放在一起

4. 定义初始化模型参数

In [95]:
# net[0] # net[0]=Linear(in_features=2, out_features=1, bias=True)
# net[0].bias是张量b，net[0].weight是张量w
net[0].weight.data.normal_(0,0.01) # 初始化
net[0].bias.data.fill_(0) # 偏差设为0

tensor([0.])

5. 使用内置的均方误差MSELoss

In [96]:
loss = nn.MSELoss()

6. 使用内置的SGD

In [97]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03) # net.parameters()是个generator包含所有的参数w和b

7. 训练过程

In [98]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y) # 如此优雅？？ net自带参数w和b
        trainer.zero_grad() # 先清零优化器的梯度
        l.backward() # pytorch自动进行了sum
        trainer.step() # 更新模型；优雅
    l = loss(net(features), labels)
    print(f'epoch {epoch+1}, train_loss {l:f}')

epoch 1, train_loss 0.000192
epoch 2, train_loss 0.000106
epoch 3, train_loss 0.000106


In [102]:
print(f'预测后的参数w: ', net[0].weight.data)
print(f'预测后的参数b: ', net[0].bias.data)

预测后的参数w:  tensor([[ 2.0007, -3.3989]])
预测后的参数b:  tensor([4.1995])
